# NSIDC ICESat2 Regression tests

### This juypter notebook runs and verifies a series of test requests against NSIDC's ICESat2 data.

Requests are submitted and the retrieved data compared to a set of verified results.

Sample requests include:

- Subset by bounding box
- Subset by temporal range
- Subset by shapefile 


We test against ICESat2 v6 collections:
[ATL03](https://nsidc.org/data/atl03/versions/6), [ATL04](https://nsidc.org/data/atl04/versions/6), [ATL06](https://nsidc.org/data/atl06/versions/6), [ATL07](https://nsidc.org/data/atl07/versions/6), [ATL08](https://nsidc.org/data/atl08/versions/6), [ATL09](https://nsidc.org/data/atl09/versions/6), [ATL10](https://nsidc.org/data/atl10/versions/6), [ATL12](https://nsidc.org/data/atl12/versions/6) and [ATL13](https://nsidc.org/data/atl13/versions/6)


## Prerequisites

The dependencies for this notebook are listed in the
[environment.yaml](https://github.com/nasa/harmony-regression-tests/blob/main/test/nsidc-icesat2/environment.yaml). To test locally, create the conda environment used in the automated regression testing suite:

```sh
conda env create -f ./environment.yaml && conda activate papermill-nsidc-icesat2
```

A `.netrc` file must also be located in the `test` directory of this repository. Additionally the credentials in this .netrc should belong to a user that can access the NSIDC data which may be protected by ACLs in UAT and SIT.


## Set the Harmony environment:

The next cell below sets the `harmony_host_url` to one of the following valid values:

* Production: <https://harmony.earthdata.nasa.gov>
* UAT: <https://harmony.uat.earthdata.nasa.gov>
* SIT: <https://harmony.sit.earthdata.nasa.gov>
* Local: <http://localhost:3000>

By default, the value is set to use Harmony's UAT environment. You can modify the target environment in two ways when using this notebook.

* Run this notebook in a local Jupyter notebook server and simply edit the value of `harmony_host_url` in the cell below to be the desired value for your environment.

* Run the `run_notebooks.sh` script, which uses the papermill library to parameterize and run notebooks. Before running, set the environment variable `HARMONY_HOST_URL` to the desired environment's URL from the list above. This variable will override the default value in the cell below, allowing papermill to inject the correct URL into the notebook at runtime.

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

### Import required packages

In [ ]:
from harmony import BBox, Client, Collection, Dimension, Environment, Request
from os.path import exists
from datetime import datetime
from pathlib import Path

#### Import shared utility functions:

In [ ]:
import sys

sys.path.append('../shared_utils')
from utilities import (
    print_error,
    print_success,
    submit_and_download,
    compare_results_to_reference_file_new,
)

### Set up test information

In [ ]:
non_production_configuration = {
    'subset_bounding_box': {
        'ATL03': {
            'collection_concept_id': Collection(id='C1256407609-NSIDC_CUAT'),
            'granule_id': 'G1262402516-NSIDC_CUAT',
            'spatial': BBox(-105.5, 40.0, -105.0, 40.005),
        },
        'ATL07': {
            'collection_concept_id': Collection(id='C1256535488-NSIDC_CUAT'),
            'granule_id': 'G1261684946-NSIDC_CUAT',
            'spatial': BBox(-112.0, 80.0, -93.0, 80.3),
        },
        'ATL08': {
            'collection_concept_id': Collection(id='C1256432189-NSIDC_CUAT'),
            'granule_id': 'G1260745539-NSIDC_CUAT',
            'spatial': BBox(-105.5, 40.0, -105.0, 40.25),
        },
    },
    'subset_by_temporal_range': {
        'ATL04': {
            'collection_concept_id': Collection(id='C1256535558-NSIDC_CUAT'),
            'granule_id': 'G1256952662-NSIDC_CUAT',
            'temporal': {
                'start': datetime.fromisoformat("2020-04-08T08:00:00.000Z"),
                'stop': datetime.fromisoformat("2020-04-08T08:05:00.000Z"),
            },
            'coords_to_rename': ['delta_time'],
        }
    },
    'subset_by_shapefile': {
        'ATL06': {
            'collection_concept_id': Collection(id='C1256358217-NSIDC_CUAT'),
            'granule_id': 'G1260779121-NSIDC_CUAT',
            'shape': 'ancillary/Iceland_sliver.geojson',
        },
    },
}


environment_configuration = {
    'https://harmony.uat.earthdata.nasa.gov': {
        **non_production_configuration,
        'env': Environment.UAT,
    },
    'https://harmony.sit.earthdata.nasa.gov': {
        **non_production_configuration,
        'env': Environment.SIT,
    },
    'http://localhost:3000': {
        **non_production_configuration,
        'env': Environment.LOCAL,
    },
}

configuration = environment_configuration.get(harmony_host_url)

if configuration is not None:
    harmony_client = Client(env=configuration['env'])

### Run Bounding Box Tests

#### TODO write what this does



In [ ]:
test_name = 'subset_bounding_box'
if configuration is not None:
    for shortname, test_config in configuration[test_name].items():
        test_request = Request(
            collection=test_config['collection_concept_id'],
            granule_id=[test_config['granule_id']],
            spatial=test_config['spatial'],
        )
        test_output = Path(f'{shortname}_{test_name}.h5')
        test_reference = Path(
            f'reference_files/{test_output.stem}_reference{test_output.suffix}'
        )

        url = harmony_client.request_as_url(test_request)
        print(f'testing URL: {url}')

        submit_and_download(harmony_client, test_request, test_output)

        assert exists(
            test_output
        ), 'Unsuccessful Harmony Request: {shortname}: {test_name}'
        compare_results_to_reference_file_new(test_output, test_reference)
        print_success(f'{shortname} {test_name} test request.')

    print_success(f'{test_name} test suite.')
else:
    print(
        f'Bounding box tests not configured for environment: {harmony_environment} - skipping tests'
    )

## Run Temporal Range Tests

In [ ]:
test_name = 'subset_by_temporal_range'
if configuration is not None:
    for shortname, test_config in configuration[test_name].items():
        test_request = Request(
            collection=test_config['collection_concept_id'],
            granule_id=[test_config['granule_id']],
            temporal=test_config['temporal'],
        )
        test_output = Path(f'{shortname}_{test_name}.h5')
        test_reference = Path(
            f'reference_files/{test_output.stem}_reference{test_output.suffix}'
        )

        url = harmony_client.request_as_url(test_request)
        print(f'testing URL: {url}')

        submit_and_download(harmony_client, test_request, test_output)

        assert exists(
            test_output
        ), 'Unsuccessful Harmony Request: {shortname}: {test_name}'
        compare_results_to_reference_file_new(
            test_output, test_reference, test_config['coords_to_rename']
        )
        print_success(f'{shortname} {test_name} test request.')

    print_success(f'{test_name} test suite.')
else:
    print(
        f'Bounding box tests not configured for environment: {harmony_environment} - skipping tests'
    )

## Run Subset by Shapefile Tests

In [ ]:
test_name = 'subset_by_shapefile'
if configuration is not None:
    for shortname, test_config in configuration[test_name].items():
        test_request = Request(
            collection=test_config['collection_concept_id'],
            granule_id=[test_config['granule_id']],
            shape=test_config['shape'],
        )
        test_output = Path(f'{shortname}_{test_name}.h5')
        test_reference = Path(
            f'reference_files/{test_output.stem}_reference{test_output.suffix}'
        )
        print(test_reference)

        submit_and_download(harmony_client, test_request, test_output)

        assert exists(
            test_output
        ), 'Unsuccessful Harmony Request: {shortname}: {test_name}'
        compare_results_to_reference_file_new(test_output, test_reference, [])
        print_success(f'{shortname} {test_name} test request.')

    print_success(f'{test_name} test suite.')
else:
    print(
        f'Bounding box tests not configured for environment: {harmony_environment} - skipping tests'
    )